<a href="https://colab.research.google.com/github/Samruddhi-saoji/Image_handler/blob/main/image_handler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob

###### read an image ####
def read(img_path):
   return cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)


##### save an image ###
def save(img, img_path="output.jpeg"):
   cv2.imwrite(img_path, img)

## normalize image before saving
def save_normalized(img, img_path="output.jpeg"):
    normalized = (img - img.min()) / (img.max() - img.min())
    uint8_img = (normalized * 255).astype(np.uint8)

    cv2.imwrite(img_path, uint8_img)


### get a list of all the images in a folder ####
# returns (list of images, list of names)
def list_images(folder_path):
    result = []

    # get list of names of all the images in the folder
    file_names = os.listdir(folder_path)

    # read each image as numpy aaray
    for img_name in file_names:
        # full path of the img = folder_path + image name
        image_path = os.path.join(folder_path, img_name)

        # Read the image and add it to the list
        img = read(image_path)
        if img is not None:
            result.append(read(image_path))

    return (result, file_names)


#delete all files in a folder
def delete_all_files(folder_path):
    files = glob.glob(os.path.join(folder_path, '*'))
    for file in files:
        if os.path.isfile(file):
            os.remove(file)


# Display image
def display(img):
    plt.imshow(img, cmap='gray')
    plt.axis('off')  # Remove axes
    plt.show()

#compress image
  # reduce img from size (r1, c1) to (r2, c2)
  # every (r1/r2)th row and every (c1/c2)th vol of the original img shld be copied to the compressed img
def compress_img(img, r1, c1, r2, c2):
    compressed_img = img[::r1 // r2, ::c1 // c2]
    return compressed_img


# (h_old, w_old) to (h_new, w_new)
def crop(img, r_start, num_rows, c_start, num_cols):
    cropped_img = img[r_start:r_start + num_rows, c_start: c_start + num_cols :]
    return cropped_img

#create a composite image from a list of images
# images = list of images
def get_composite(images, weights=[], sum_only=False):
    # Ensure all images have the same shape
    image_shape = images[0].shape
    for img in images:
        if img.shape != image_shape:
            raise ValueError("All images must have the same shape")

    # Initialize an array to store the composite image
    composite_image = np.zeros_like(images[0], dtype=np.float32)

    n = len(images)
    # if weights value has not been passed
    if len(weights)==0:
        weights = [1 for _ in range(n)]

    if len(weights) != n:
        print("No of images and weights should be same")
        return -1

    # Sum the pixel values from all images
    for i in range(n):
        img = images[i]
        w = weights[i]
        composite_image = composite_image + w*img.astype(np.float32)
        np.clip(composite_image, 0, 255)

    if sum_only == False:
        composite_image = (composite_image/sum(weights))#(composite_image / len(images)) #.astype(np.uint8)

    composite_image.astype(np.uint8)
    return np.clip(composite_image, 0, 255)





# Coloured image handling (BGR)

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import glob

#### Reading and Saving #####
def read(img_path):
   return cv2.imread(img_path, cv2.IMREAD_COLOR)

def read_grayscale(img_path):
   return cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

def save(img, img_path="output.jpeg"):
   cv2.imwrite(img_path, img)


# returns (list of images, list of names)
def list_images(folder_path):
    result = []

    # get list of names of all the images in the folder
    file_names = os.listdir(folder_path)

    # read each image as numpy aaray
    for img_name in file_names:
        # full path of the img = folder_path + image name
        image_path = os.path.join(folder_path, img_name)

        # Read the image and add it to the list
        img = read(image_path) # read as (r,g,b)
        if img is not None:
            result.append(read(image_path))

    return (result, file_names)


#delete all files in a folder
def delete_all_files(folder_path):
    files = glob.glob(os.path.join(folder_path, '*'))
    for file in files:
        if os.path.isfile(file):
            os.remove(file)


#### Updating pixel colour ####
# Update pixel at (r, c) to new_clr
# new_clr = (b, g, r)
def set_pixel(img, pix, new_clr):
    r, c = pix[0], pix[1]
    b_new, g_new, r_new = new_clr[0], new_clr[1], new_clr[2]

    #update the pixel
    img.itemset((r, c, 0), b_new)  # Blue channel
    img.itemset((r, c, 1), g_new)  # Green channel
    img.itemset((r, c, 2), r_new)  # Red channel

    #return updated image
    return img

def set_blue(img, pix, new_blue):
    img.itemset((pix[0], pix[1], 0), new_blue)  # Blue channel
    return img

def set_green(img, pix, new_green):
    img.itemset((pix[0], pix[1], 1), new_green)
    return img

def set_red(img, pix, new_red):
    img.itemset((pix[0], pix[1], 2), new_ref)
    return img

#### Check colour ####
# returns True if given pixel is red
def isRed(pixel):
   b, g, r = pixel
   return r > 200 and g < 100 and b < 100

def isBlue(pixel, thresh=100):
   b, g, r = pixel
   #if (b-g)>=thresh and (b-r)>=thresh: return True
   #return False
   return b > 200 and g < 100 and r < 100

def isGreen(pixel):
   b, g, r = pixel
   #if (g-b)>=75 and (g-r)>=75: return True
   #return False
   return g > 200 and r < 100 and b < 100


# if approx is True, then nearly gray is also ok
def isGray(pixel, approx=False, thresh=20):
   b, g, r = pixel # image is gray if b=g=r

   if not approx:
      if (b == g) and (g == r):
         return True
      else: return False

   else: # approximately gray is ok
      return abs(r - g) < thresh and abs(g - b) < thresh


# if approx is True, then nearly gray is also ok
def isWhite(pixel, approx=True, thresh=200):
   b, g, r = pixel

   if not approx:
      if b!=255 or g!=255 or r!=255: return False
      else: return True

   else: # approximately white is ok
      return r > thresh and g > thresh and b > thresh


def isBlack(pixel, approx=True, thresh=200):
   b, g, r = pixel

   if not approx:
      if b!=0 or g!=0 or r!=0: return False
      else: return True

   else: # approximately black is ok
      return r < thresh and g < thresh and b < thresh


#### basic operations ####
#compress image
  # reduce img from size (r1, c1) to (r2, c2)
  # every (r1/r2)th row and every (c1/c2)th vol of the original img shld be copied to the compressed img
def compress_img(img, r1, c1, r2, c2):
    compressed_img = img[::r1 // r2, ::c1 // c2]
    return compressed_img

# (h_old, w_old) to (h_new, w_new)
def crop(img, r_start, num_rows, c_start, num_cols):
    cropped_img = img[r_start:r_start + num_rows, c_start: c_start + num_cols :]
    return cropped_img

# Others

### Mark points p1, p2, .. pn on a grayscale image

#### Show only (using matplotlib)

In [ ]:
# points = list of points p(x, y)
# x coord --> col
# y coord --> row
def show_points_mpl(img, points):
    # initialise output image
    output = np.copy(img)

    # change each point to white
    for point in points:
        r, c = point[1], point[0]
        output[r, c] = 255
    # red marker points will be better visible on white

    # Display the modified image
    plt.imshow(output, cmap='gray') #grayscale img
    # Mark all points with a red dot
    plt.scatter([point[0] for point in points], [point[1] for point in points],
                color='red', marker='o', s=2)
    plt.axis('off') # Remove axes
    plt.show()

#### Mark points and save image (using cv2)

In [ ]:
# img = grayscale image
# points = list of points p(x, y)
  # x coord --> col
  # y coord --> row
# dest_path = path to save the image
def show_points(img, points, dest_path):
    # Convert grayscale image to color (BGR format)
    color_image = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

    # Draw red circles (points) on the color image
    for point in points:
        cv2.circle(color_image, point, radius=0, color=(0, 0, 255), thickness=0)  # Red color

    # Save the updated image
    cv2.imwrite(dest_path, color_image)

### Cropping images

In [ ]:
# w_old >= w_new
# start from 1st row, 1st col
def crop(img, w_new, h_new):
    # old dimensions
    h_old, w_old = img.shape[:2]

    x_start = (w_old - w_new) // 2
    y_start = (h_old - h_new) // 2
    x_end = x_start + w_new
    y_end = y_start + h_new

    cropped_img = img[y_start:y_end, x_start:x_end]
    return cropped_img



# keep only n number of rows, starting from the rth row
# thus, keep rows r to (r+n-1)
def crop_height(img, r, n):
    cropped_img = img[r:r + n, :]
    return cropped_img

### TIF to jpeg conversion

In [ ]:
############# TIF to JPEG conversion ################

from PIL import Image

def convert_tif_to_jpeg(src, dest):
  """
  Converts a TIFF file to JPEG format using Pillow (PIL Fork).

  Args:
      src (str): Path to the source TIFF file.
      dest (str): Path to the destination JPEG file.
  """
  try:
    # Open the TIFF file
    img = Image.open(src)

    # Convert the image to RGB mode if necessary
    if img.mode != 'RGB':
      img = img.convert('RGB')

    # Save the image as JPEG with desired quality (0-95)
    img.save(dest, "JPEG", quality=90)
    print(f"Converted TIFF '{src}' to JPEG '{dest}'.")
  except FileNotFoundError:
    print(f"Error: File '{src}' not found.")
  except OSError:
    print(f"Error: Could not convert '{src}' to JPEG.")